In [1]:
from perceval.backends.core.github import GitHub
from datetime import date
import datetime
from datetime import datetime as dt
import dateutil.tz
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy as s
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker
import augur
from time import gmtime, strftime

## Load

First loads the users data from 'augur.config.json' so will take the Database information (e.g. name of database, port of database). Then connects to the database using augur.App.github_issues() and takes the details of the database and connects to to the database using charset 'utf8'. It also makes a connection to piper_reader

In [2]:
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
connect,list1,path = augurApp.github_issues()
token = list1[5]
DB_STR = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(
            list1[0], list1[1], list1[2],\
            list1[3], list1[4]
        )
db = s.create_engine(DB_STR)
Piper, path= augurApp.piper()

## Connect:

Queries what tables are in the database and determines if 'github_issues_2' is there if it is then 'git_repos' is set as 'True', if it isn't there 'git_repos' is set as 'False'. Then we determine what git repositories are in 'github_issues_repo_jobs' and determine how many rows are in it. If 'github_issues_repo_jobs' is not in the database it is created and the column 'augurlistID' is set as the primary key. We then add a connection to 'github_issues_repo_jobs' so that we can change the column 'last_run' if new messages were downloaded for a git repository.

In [3]:
table_names = s.inspect(db).get_table_names()
print(table_names)
val = False
#mail_lists = True
git_repos = True
numb = 0
item = 1
columns2 = "augurlistID","backend_name","github_repo_url","project","last_run"
df_mail_list = pd.DataFrame(columns=columns2)
if("github_issues_first_message" not in table_names):
    #mail_lists = False
    git_repos = False
else:
    SQL = s.sql.text("""SELECT COUNT(*) FROM github_issues_first_message""")
    df7 = pd.read_sql(SQL, db)
    augurmsgID = int(df7.values)+1
    item = augurmsgID
if("github_issues_repo_jobs_first_message" in table_names):
    lists_createdSQL = s.sql.text("""SELECT project FROM github_issues_repo_jobs_first_message""")
    df1 = pd.read_sql(lists_createdSQL, db)
    print(df1)
    val = True
    val = db.engine.execute("""SELECT augurlistID FROM 
                                   github_issues_repo_jobs_first_message
                                   ORDER BY augurlistID DESC LIMIT 1""")
    for i in val:
        numb = i['augurlistID']
    
else:
    df_mail_list.to_sql(name="github_issues_repo_jobs_first_message",con=db,if_exists='replace',index=False,
                        dtype={'augurlistID': s.types.Integer(),
                               'backend_name': s.types.VARCHAR(length=300),
                               'github_repo_url': s.types.VARCHAR(length=300),
                               'project': s.types.VARCHAR(length=300),
                               'last_run': s.types.DateTime()
                        })
    lists_createdSQL = s.sql.text("""SELECT project FROM github_issues_repo_jobs_first_message""")
    df1 = pd.read_sql(lists_createdSQL, db)
    db.execute("ALTER TABLE github_issues_repo_jobs_first_message ADD PRIMARY KEY (augurlistID)")
    print("Created Table")
Base = declarative_base(db)
class table(Base):
    __tablename__ = 'github_issues_repo_jobs_first_message'
    __table_args__={'autoload':True}
    
Session = sessionmaker(bind=db)
session = Session()
res = session.query(table).all()

['github_issues', 'github_issues_2', 'github_issues_repo_jobs', 'github_issues_sentiment_scores', 'github_issues_sentiment_scores_senticr', 'github_pull_request_repo_jobs', 'github_pull_requests', 'github_pull_requests_2', 'github_pull_requests_sentiment_scores', 'github_pull_requests_sentiment_scores_2', 'github_pull_requests_sentiment_scores_senticr', 'issue_response_time', 'issue_response_time2', 'issue_response_time_3', 'mail_lists', 'mail_lists_sentiment_scores_nltk', 'mail_lists_sentiment_scores_senticr', 'mailing_list_jobs', 'profane_words']
Created Table


## Write to file:

Determines if the file with the git repositories were created, if not it writes a set of default git repositories (to show how the program would work).

In [4]:
if "notebooks" in os.getcwd():
    os.chdir("..")
connect,list1,path = augurApp.github_issues()
print(path)
if(not os.path.exists(path)):
    file = open(path, "w+")
else:
    file = open(path, "r")
    print("yeah")
if (os.stat(path).st_size == 0):
    file.write("owner,repo_url\n")
    file.write("chaoss,\"whitepaper\"\n")
    file.write("chaoss,\"prospector\"\n")
    #file.write("https://lists.opendaylight.org/pipermail/,\"archetypes-dev\"\n")
    print("Please enter the mailing lists and the links for them please")
    print("Going to the default mailing lists")

count = 0
for line in file:
    print(line)
    count+=1
    if(count == 2):
        break
file.close()

print(df1['project'].values)

runtime/git_repos.csv
yeah
owner,repo_url

nodejs,"CTC"

[]


## Iteration through github repositories

This first reads the git repositories stored in 'path' and we also get the current date 'today'. We then create a dataframe 'github_issues_2' that will store all the comments under an issue and the issue itself. Grouping the issues by owner we iterate through the different repositories and then iterate over the messages for a particular issue. We then pull the  data from GitHub and we determine if the repository is stored in 'github_issues_repo_jobs', if it is we set 'new' to 'True' and store that repository so it can be added to the table 'github_issues_repo_jobs and 'froms' is set as 'None'. If it is in 'github_issues_repo_jobs' we set today's date to 'last_run' for the repository in 'github_issues_repo_jobs' and 'froms' is set as today's date.

We then go about downloading the github issues for a particular repository, the 'from_date' is set as 'froms'. We then convert issues to a format that can be uploaded to the datebase. If the git repository is stored in 'github_issues_repo_jobs' then only the issues that occur after the date stored in 'last_run' is uploaded. If the git repository isn't in the database all the issues are uploaded. We finally upload the git repositories information to the table 'github_issues_repo_jobs'. Since github has a limit as to how much data can be downloaded, if a large repository with a number of issues is being downloaded the program will run for hours, since the Rate limit must be reset which takes around an hour each time.

In [5]:
'''
owner,repo_url
torvalds,"linux"
nodejs,"CTC"
nodejs,"TSC"
'''
'''
owner,repo_url
chaoss,"whitepaper"
chaoss,"prospector"
'''
k=0
df5 = pd.read_csv(path)
today = strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime())
#for issues in repo.fetch(from_date=date):
columns1 = 'augurmsgID',"backend_name",'repo_link',"owner","repo","subject",\
          "status","category","issue_number","timestamp",\
          "issue_id","user","body"
df = pd.DataFrame(columns=columns1)
df.to_sql(name="github_issues_first_message", con=db,\
          if_exists='append',index=False,
           dtype={'augurmsgID': s.types.Integer,#0
                  'backend_name': s.types.VARCHAR(length=300),#1
                  'repo_link': s.types.VARCHAR(length=300),#2
                  'owner': s.types.VARCHAR(length=300),#3
                  'repo': s.types.VARCHAR(length=300),#4
                  'subject': s.types.VARCHAR(length=300),#5
                  'status': s.types.VARCHAR(length=10),#6
                  'category': s.types.VARCHAR(length=1000),#7
                  'issue_number': s.types.Integer,#8
                  'date': s.types.DateTime(),#9
                  'issue_id': s.types.Integer,#10
                  'user': s.types.VARCHAR(length=100),#11
                  'body':s.types.TEXT#12              
           })

groups = df5.groupby('owner').groups
x = 0
new = False
for group in groups:
    own = group
    repo_url = (df5.loc[df5['owner'] == group]['repo_url']).tolist()
    print(own + '/' + repo_url[0])
    #break
    for y in repo_url:        
        repo = GitHub(owner=own,repository=y,api_token=token,\
                      sleep_for_rate=True,min_rate_to_sleep=500)
        inside = own + '/' + y
        if(inside not in df1['project'].values):
            new = True
            li = [[numb,"GitHub",'https://github.com/' + inside, inside,Piper.convert_date(today)]]
            df8 = pd.DataFrame(li,columns=columns2)
            df4 = df_mail_list.append(df8)
            df_mail_list = df4
            numb+=1
            froms = None
            #continue
        else:
            j = 0
            #print(today)
            time = Piper.convert_date(today)
            while(res[j].project!=inside):
                #print(res[j].project)
                j+=1
            now = res[j].last_run
            print(now)
            res[j].last_run = time
            session.commit()
            froms = now
            #froms = Piper.convert_date(now)
            #continue
            #"Thu, 5 Mar 2018 14:57:56 +0000"
        for issues in repo.fetch(from_date = froms):
            if 'pull_request' in issues['data']:
                #print(issues['data']['number'])
                continue
            else: 
                created = issues['data']['created_at']
                #print(type(froms))
                issue_time = created[:10] + " " + created[11:19]
                issue_time = dt.strptime(issue_time,"%Y-%m-%d %H:%M:%S")
                text = issues['data']['body']
                user = issues['data']['user']['login']
                num = issues['data']['number']
                id_num = issues['data']['id']
                store = [item,issues['backend_name'],issues['tag'],own,y,\
                         issues['data']['title'],issues['data']['state'],\
                         issues['category'],num,issue_time,\
                         id_num,user,text]
                df_user = pd.DataFrame([store],columns=columns1)
                if(froms == None):
                    df = df.append(df_user)
                    item+=1
                elif(issue_time > froms):
                    df = df.append(df_user)
                    item+=1
                    

            if(df.shape[0] > 500):
                db = s.create_engine(DB_STR)
                df.to_sql(name='github_issues_first_message', con=db,\
                          if_exists='append',index=False)
                df = pd.DataFrame(columns=columns1)
                print("Broken")
                #break
            if(k%50 == 0):
                print(k)
            k+=1
            #if(k == 2):
            #    break
        
        if(df.shape[0] < 500):
            db = s.create_engine(DB_STR)
            df.to_sql(name='github_issues_first_message', con=db,\
                          if_exists='append',index=False)
            df = pd.DataFrame(columns=columns1)
            print("Broken")
        print("Total Number of issues: ", k)
        print(df.shape[0])
if(new == True):
    df_mail_list.to_sql(name='github_issues_repo_jobs_first_message',con=db,if_exists='append',index=False)

nodejs/CTC


2019-04-23 23:36:28 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/rvagg
2019-04-23 23:36:29 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/vkurchatkin
2019-04-23 23:36:29 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/benjamingr
2019-04-23 23:36:30 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/Trott
2019-04-23 23:36:30 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/joshgav
2019-04-23 23:36:31 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/Fishrock123


0


2019-04-23 23:36:32 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/jasnell
2019-04-23 23:36:32 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/addaleax
2019-04-23 23:36:33 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/ChALkeR
2019-04-23 23:36:33 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/bnoordhuis
2019-04-23 23:36:34 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/mhdawson
2019-04-23 23:36:34 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/williamkapke
2019-04-23 23:36:35 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/ofrobots
2019-04-23 23:36:36 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/sam-github
2019-04-23 23:36:36 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/u

2019-04-23 23:37:18 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/zkat
2019-04-23 23:37:21 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/i5ting
2019-04-23 23:37:22 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/pesho
2019-04-23 23:37:23 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/thekemkid
2019-04-23 23:37:23 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/heapwolf
2019-04-23 23:37:24 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/lamanabie
2019-04-23 23:37:25 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/CrabDude
2019-04-23 23:37:25 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/saschanaz
2019-04-23 23:37:25 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/leodutr

...

2019-04-24 03:55:00 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/ozziest
2019-04-24 03:55:01 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/TimothyYe
2019-04-24 03:55:01 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/imroc
2019-04-24 03:55:02 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/midnio
2019-04-24 03:55:05 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/adoyle-h
2019-04-24 03:55:06 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/lamcw
2019-04-24 03:55:07 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/chloecaelynn
2019-04-24 03:55:07 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/onstring
2019-04-24 03:55:08 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/Solex

2019-04-24 03:55:43 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/ziishaned
2019-04-24 03:55:44 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/SandroRybarik
2019-04-24 03:55:44 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/hollowbit
2019-04-24 03:55:44 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/daneshk
2019-04-24 03:55:45 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/congnd
2019-04-24 03:55:46 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/SafronovVV
2019-04-24 03:55:46 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/fozbek
2019-04-24 03:55:46 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/serbaniuliuscezar
2019-04-24 03:55:47 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.githu

2019-04-24 03:56:34 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/beacya
2019-04-24 03:56:34 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/loopdigga98
2019-04-24 03:56:34 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/liyiheng1111
2019-04-24 03:56:35 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/s0md3v
2019-04-24 03:56:36 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/yysbigsong
2019-04-24 03:56:37 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/schampions
2019-04-24 03:56:37 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/georgmay
2019-04-24 03:56:38 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/haxzie
2019-04-24 03:56:38 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/us

2019-04-24 03:57:13 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/GzimH
2019-04-24 03:57:13 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/ertanhasani
2019-04-24 03:57:14 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/Marco98
2019-04-24 03:57:14 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/jatovar
2019-04-24 03:57:15 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/devcrack
2019-04-24 03:57:15 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/mesquita
2019-04-24 03:57:15 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/CaptainFreak
2019-04-24 03:57:16 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/users/martincarles
2019-04-24 03:57:17 keanu-Inspiron-5567 root[32000] INFO Getting info for https://api.github.com/u

Broken
Total Number of issues:  548
0
